In [56]:
#!value --name strategyGuide --from-file strategy.txt

In [57]:
using System;

readonly record struct OpponentChoice(string Value)
{
   public static implicit operator string(OpponentChoice choice) => choice.Value;
   public static implicit operator OpponentChoice(string value) => new(value);
}

readonly record struct PredictedChoice(string Value)
{
   public static implicit operator string(PredictedChoice choice) => choice.Value;
   public static implicit operator PredictedChoice(string value) => new(value);
}

readonly record struct StrategyChoice(string Value)
{
   public static implicit operator string(StrategyChoice choice) => choice.Value;
   public static implicit operator StrategyChoice(string value) => new(value);
}

readonly record struct RoundStrategy(OpponentChoice opponent, StrategyChoice strategy);
readonly record struct Round(OpponentChoice opponent, PredictedChoice prediction);

Func<PredictedChoice, int> scorePrediction = p => p.Value switch
{
    "X" => 1,
    "Y" => 2,
    "Z" => 3,
    _ => -1
};

Func<OpponentChoice, PredictedChoice> win = o => o.Value switch
{
    "A" => new ("Y"),
    "B" => new ("Z"),
    "C" => new ("X"),
    _ => new ("?")
};

Func<OpponentChoice, PredictedChoice> lose = o => o.Value switch
{
    "A" => new ("Z"),
    "B" => new ("X"),
    "C" => new ("Y"),
    _ => new ("?")
};

Func<OpponentChoice, PredictedChoice> draw = o => o.Value switch
{
    "A" => new ("X"),
    "B" => new ("Y"),
    "C" => new ("Z"),
    _ => new ("?") 
};

Func<OpponentChoice, Func<PredictedChoice, int>> scoreHand = o => p => o.Value switch
{
    "A" when p.Value == "X" => 3,
    "B" when p.Value == "Y" => 3,
    "C" when p.Value == "Z" => 3,
    "A" when p.Value == "Z" => 0,
    "B" when p.Value == "X" => 0,
    "C" when p.Value == "Y" => 0,
    "C" when p.Value == "X" => 6,
    "A" when p.Value == "Y" => 6,
    "B" when p.Value == "Z" => 6,
    _ => -1
};

Func<OpponentChoice, Func<PredictedChoice, int>> totalScoreForRound = o => p =>
    scorePrediction(p) + scoreHand(o)(p);

Func<StrategyChoice, Func<OpponentChoice, PredictedChoice>> part1Strategy = s => _ => new PredictedChoice(s.Value);
Func<StrategyChoice, Func<OpponentChoice, PredictedChoice>> part2Strategy = c => c.Value switch
{
    "X" => lose,
    "Y" => draw,
    "Z" => win,
    _ => lose
};

var buildRounds = (string strategyGuide) =>
    (Func<StrategyChoice, Func<OpponentChoice, PredictedChoice>> selectionStrategy) =>
        strategyGuide
            .Split(Environment.NewLine)
            .Select(s =>
            {
                OpponentChoice opponent = s.Split(' ').First();
                StrategyChoice strategy = s.Split(' ').Skip(1).First();
                return new RoundStrategy(opponent, strategy);
            })
            .Select(r => new Round(r.opponent, selectionStrategy(r.strategy)(r.opponent)));

var scoreGame = (IEnumerable<Round> rounds) =>
    rounds
        .Select(r => totalScoreForRound(r.opponent)(r.prediction))
        .Sum();

In [58]:
// proof on sample data

var sampleStrategy =
"""
A Y
B X
C Z
""";

var rounds = buildRounds(sampleStrategy)(part1Strategy);
var totalScore = scoreGame(rounds);
totalScore == 15

True

In [59]:
#!share strategyGuide --from value

// part 1
var rounds = buildRounds(strategyGuide)(part1Strategy);
var totalScore = scoreGame(rounds);
totalScore

14069

In [60]:
// part 2 proof on sample data

var rounds = buildRounds(sampleStrategy)(part2Strategy);
var totalScore = scoreGame(rounds);
totalScore == 12

True

In [61]:
#!share strategyGuide --from value

// part 2
var rounds = buildRounds(strategyGuide)(part2Strategy);
var totalScore = scoreGame(rounds);
totalScore

12411